In [236]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2022-ai-w7p2/sample_submission.csv
/kaggle/input/2022-ai-w7p2/x_test.csv
/kaggle/input/2022-ai-w7p2/y_train.csv
/kaggle/input/2022-ai-w7p2/x_train.csv


In [237]:
import torch
import random

In [238]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
  torch.cuda.manual_seed_all(777)

In [239]:
#데이터 가져오기
import numpy as np 
import pandas as pd

#index_col=0으로 인덱스 열 제거
x_train = pd.DataFrame(pd.read_csv('/kaggle/input/2022-ai-w7p2/x_train.csv',engine='python',encoding='CP949'))
x_train

,시도명,읍면동명,거주인구,근무인구,방문인구,총 유동인구,평균 속도,평균 소요 시간,평균 기온,일강수량,평균 풍속,월,일,년
0,0,26,76018.965,5009.811,36887.341,117916.117,40.467,21.667,17.750,0.000,2.725,9,25,2019
1,1,24,1232416.968,76043.955,451558.268,1760019.191,38.126,31.632,13.300,14.500,4.500,3,4,2018
2,1,23,134260.946,8412.761,78177.981,220851.688,51.107,45.643,5.250,0.500,4.950,4,7,2018
3,1,25,289327.429,39102.424,287029.303,615459.156,33.067,48.000,26.300,0.500,1.500,8,14,2019
4,1,7,1116617.660,96560.651,524905.301,1738083.612,40.392,26.961,8.400,44.000,4.300,12,1,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21481,0,6,91977.598,14802.819,79885.446,186665.863,41.676,21.676,6.400,0.000,1.600,3,5,2020
21482,0,9,63673.610,3725.663,65813.385,133212.658,48.737,26.105,18.300,7.000,3.700,5,26,2018
21483,1,3,406035.642,46335.550,258079.137,710450.329,49.314,50.943,-6.133,0.833,6.067,1,25,2018
21484,1,25,315737.894,25379.224,283079.548,624196.666,35.312,43.688,8.600,0.000,3.100,3,17,2019


In [240]:
y_train = pd.DataFrame(pd.read_csv('/kaggle/input/2022-ai-w7p2/y_train.csv',engine='python',encoding='CP949'))
y_train

,교통량
0,0.000
1,1532.989
2,710.107
3,374.200
4,535.882
...,...
21481,91.000
21482,0.000
21483,0.000
21484,283.625


In [241]:
x_test = pd.DataFrame(pd.read_csv('/kaggle/input/2022-ai-w7p2/x_test.csv',engine='python',encoding='CP949'))
x_test

,시도명,읍면동명,거주인구,근무인구,방문인구,총 유동인구,평균 속도,평균 소요 시간,평균 기온,일강수량,평균 풍속,월,일,년
0,0,28,45737.828,8703.034,91775.025,146215.887,57.286,35.000,6.500,0.0,2.9,1,12,2020
1,1,21,289284.270,48392.749,179649.272,517326.291,50.500,58.912,18.833,0.0,2.3,6,10,2019
2,1,4,72684.218,45632.065,120931.098,239247.380,34.727,60.000,13.200,0.0,1.8,5,10,2019
3,1,12,235752.513,34212.805,186910.679,456875.997,37.167,26.500,11.700,0.0,4.8,3,19,2020
4,1,17,544250.956,64188.293,205182.677,813621.925,31.462,37.769,22.233,0.0,2.3,9,5,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10578,1,37,30949.221,7243.292,73243.733,111436.245,27.500,26.000,0.100,1.0,2.7,2,9,2019
10579,0,39,287544.537,19901.138,230494.219,537939.894,49.130,55.130,20.600,9.0,4.3,7,8,2018
10580,1,35,217597.813,18838.618,150540.187,386976.617,27.500,30.750,0.000,0.0,5.8,2,12,2018
10581,0,6,80350.674,26339.878,101347.505,208038.057,41.000,19.968,18.800,1.0,2.4,10,18,2019


In [242]:
#데이터 표준화
# x_train=x_train.iloc[:,2:-3]
# x_test=x_test.iloc[:,2:-3]
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()

x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)

x_train = torch.FloatTensor(x_train).to(device)
y_train = torch.FloatTensor(y_train.values).to(device)
x_test = torch.FloatTensor(x_test).to(device)

print(x_train)
print(y_train)
print(x_test)

tensor([[-1.2157,  0.5220, -0.8079,  ...,  0.8355,  1.0512,  0.3829],
        [ 0.8226,  0.3532,  3.0638,  ..., -0.8805, -1.3328, -1.0358],
        [ 0.8226,  0.2688, -0.6129,  ..., -0.5945, -0.9923, -1.0358],
        ...,
        [ 0.8226, -1.4192,  0.2970,  ..., -1.4524,  1.0512, -1.0358],
        [ 0.8226,  0.4376, -0.0053,  ..., -0.8805,  0.1430,  0.3829],
        [ 0.8226,  0.6064,  2.5941,  ...,  1.1215, -1.6734,  0.3829]],
       device='cuda:0')
tensor([[   0.0000],
        [1532.9890],
        [ 710.1070],
        ...,
        [   0.0000],
        [ 283.6250],
        [ 254.1880]], device='cuda:0')
tensor([[-1.2157,  0.6908, -0.9092,  ..., -1.4524, -0.4246,  1.8015],
        [ 0.8226,  0.1000, -0.0938,  ..., -0.0225, -0.6517,  0.3829],
        [ 0.8226, -1.3348, -0.8190,  ..., -0.3085, -0.6517,  0.3829],
        ...,
        [ 0.8226,  1.2816, -0.3338,  ..., -1.1665, -0.4246, -1.0358],
        [-1.2157, -1.1660, -0.7934,  ...,  1.1215,  0.2565,  0.3829],
        [ 0.8226, -1.0

In [243]:
# 학습 파라미터 설정
learning_rate = 0.01
training_epochs = 50
batch_size = 100
drop_prob = 0.3

In [244]:
#데이터 로더 설정
from torch.utils.data import DataLoader, TensorDataset

train_dataset=TensorDataset(x_train,y_train)
train_loader = DataLoader(dataset=train_dataset,batch_size=batch_size)

In [245]:
linear1 = torch.nn.Linear(14,512,bias=True)
linear2 = torch.nn.Linear(512,512,bias=True)
linear3 = torch.nn.Linear(512,512,bias=True)
linear4 = torch.nn.Linear(512,512,bias=True)
linear5 = torch.nn.Linear(512,1,bias=True)
relu = torch.nn.ReLU()
dropout = torch.nn.Dropout(p=drop_prob)

In [246]:
torch.nn.init.xavier_normal_(linear1.weight)
torch.nn.init.xavier_normal_(linear2.weight)
torch.nn.init.xavier_normal_(linear3.weight)
torch.nn.init.xavier_normal_(linear4.weight)
torch.nn.init.xavier_normal_(linear5.weight)

Parameter containing:
tensor([[-3.6713e-02,  6.7380e-03, -5.2585e-03, -1.7382e-02,  2.8180e-02,
          1.0887e-01, -2.4962e-02,  1.1782e-01, -1.6760e-02,  4.2989e-02,
          3.8647e-02,  1.4685e-02,  1.0353e-01, -4.7054e-02,  6.9840e-02,
         -4.7351e-02, -6.5643e-02,  4.2353e-02, -5.7360e-02,  5.0114e-02,
         -2.7609e-02,  1.1161e-01, -1.4944e-01, -4.9749e-02, -6.7113e-03,
          2.0931e-02, -6.8532e-02, -9.9546e-02,  6.7625e-02, -1.8560e-02,
          1.2371e-02,  3.5323e-02,  7.0682e-02,  8.7220e-02, -7.0504e-02,
         -1.2751e-01,  1.7246e-02, -7.1924e-03, -9.9854e-02,  1.0234e-01,
          1.1197e-01,  1.7737e-02, -2.4713e-02, -4.5214e-02, -6.6245e-02,
         -3.3088e-02,  2.4879e-02,  8.8858e-02, -8.6008e-02,  5.3048e-02,
          5.5597e-02, -2.9830e-02, -9.3625e-02, -3.2320e-02, -1.7316e-02,
         -3.4877e-02,  4.7657e-02, -1.6106e-02,  3.2903e-03,  3.9246e-02,
          8.1403e-02, -2.0843e-02, -7.2599e-02,  3.8420e-02,  6.0377e-02,
          1.9045

In [247]:
#모델 선언
model = torch.nn.Sequential(linear1,relu,dropout,
                            linear2,relu,dropout,
                            linear3,relu,dropout,
                            linear4,relu,dropout,
                            linear5).to(device)

In [248]:
#손실함수와 최적화 함수 선언
#분류 문제는 CrossEntropy,회귀는 MSE
loss = torch.nn.MSELoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 

In [249]:
total_batch = len(train_loader)
model.train()

for epoch in range(training_epochs):
    avg_cost = 0

    for x_train_batch, y_train_batch in train_loader:

        optimizer.zero_grad()

        x_train_batch = x_train_batch.to(device)
        y_train_batch = y_train_batch.to(device)

        hypothesis = model(x_train_batch)
        cost = loss(hypothesis, y_train_batch)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    if epoch%10==0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 146962.656250000
Epoch: 0011 cost = 28327.871093750
Epoch: 0021 cost = 25505.724609375
Epoch: 0031 cost = 22750.775390625
Epoch: 0041 cost = 20974.494140625
Learning finished


In [250]:
# 학습된 모델로 test 데이터 예측
with torch.no_grad():
    model.eval()

    prediction = model(x_test)
    print(prediction)

tensor([[ 16.6530],
        [ 17.1006],
        [ 15.6789],
        ...,
        [ 19.3566],
        [ 13.1498],
        [642.3717]], device='cuda:0')


In [251]:
# csv 변환
prediction = prediction.cpu().numpy()
submit = pd.read_csv("/kaggle/input/2022-ai-w7p2/sample_submission.csv")
submit['predict'] = prediction
submit.to_csv('submission.csv',index=False)